# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de grandes volúmenes de datos (Gpo 10)**

### Tecnológico de Monterrey

## Actividad 3

###  Aprendizaje supervisado y no supervisado


#### **Nombrey matrícula**

*   **A01746998** - Alexys Martín Coate Reyes


# **1. Introducción teórica**



## **Aprendizaje Supervisado**

El aprendizaje supervisado intenta predecir un output con base en una serie de datos de entrenamiento que vienen previamente clasificados de manera correcta. Este tipo de aprendizaje se asemeja al que tiene un niño pequeño que está aprendiendo a abstraer las características por medio de una guía, padre o profesor.

Entre los algoritmos supervisados más populares están los siguientes:

* Regresión Lineal
  * ***pyspark.ml.regression.LinearRegression***
* Regresión Logística
  * ***pyspark.ml.classification.LogisticRegression***
* Árboles de Decisión
  * ***pyspark.ml.classification.DecisionTreeClassifier***
  * ***pyspark.ml.regression.DecisionTreeRegressor***
* Bosques Aleatorios (Random Forests)
  * ***pyspark.ml.classification.RandomForestClassifier***
  * ***pyspark.ml.regression.RandomForestRegressor***
* Máquinas de Soporte Vectorial (SVM)
  * ***pyspark.ml.classification.LinearSVC***
* Gradient-Boosted Trees (GBTs)
  * ***pyspark.ml.classification.GBTClassifier***
  * ***pyspark.ml.regression.GBTRegressor***
* Bayes Ingenuo (Naive Bayes)
  * ***pyspark.ml.classification.NaiveBayes***

# **2. Selección de los datos**

### Cargando los datos

In [ ]:
# Librerias
from pyspark.sql import SparkSession

In [ ]:
# Crear sesión Spark
spark = SparkSession.builder \
    .appName("EDA_Vuelos") \
    .getOrCreate()

# Leer el CSV
df = spark.read.csv("./Airline_Delay_2016-2018.csv", header=True, inferSchema=True)

# Mostrar esquema de datos
df.printSchema()

# Número total de registros
total_registros = df.count()
print(f"Número total de registros: {total_registros}")

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

Número total de registros: 296791


In [ ]:
# Imprimiendo los 3 primeros rengloes del dataframe dataframe
df.show(3)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2016-01-01|        DL|             1248|   DTW| LAX|        1935|  1935.0|      0.0|    23.0|    1958.0|   2107.0|   13.0|        2144|  2120.0|    -24.0|      0.0|     0.0|           309.0|              285.0|   249.0|  1979.0|
|2016-01-01|        DL|             1251|   ATL| GRR|        2125|  2130.0|     

### Seleccionando variables de caracterización

In [ ]:
# Variables seleccionadas
vars_particion = ["OP_CARRIER", "ORIGIN", "DEST", "CANCELLED", "DIVERTED"]

# Imrpimiendo la cantidad de valores únicos que se tiene por las columnas seleccionadas
for col in vars_particion:
    print(col, df.select(col).distinct().count())

OP_CARRIER 12
ORIGIN 294
DEST 294
CANCELLED 3
DIVERTED 3


In [ ]:
from pyspark.sql.functions import col, count, round

# Calcular frecuencia de combinaciones
combinaciones = df.groupBy(vars_particion).count()

# Total de registros
total = df.count()

# Agregar probabilidad
combinaciones = combinaciones.withColumn("probabilidad", col("count") / total)

combinaciones.show(truncate=False)

+----------+------+----+---------+--------+-----+---------------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |
+----------+------+----+---------+--------+-----+---------------------+
|EV        |ATL   |AGS |0.0      |0.0     |98   |3.301986920088547E-4 |
|EV        |DTW   |BWI |0.0      |0.0     |4    |1.3477497633014479E-5|
|EV        |BRO   |IAH |0.0      |0.0     |60   |2.0216246449521717E-4|
|F9        |DEN   |MIA |0.0      |0.0     |21   |7.075686257332601E-5 |
|WN        |BDL   |RSW |0.0      |0.0     |21   |7.075686257332601E-5 |
|WN        |BWI   |MEM |0.0      |0.0     |21   |7.075686257332601E-5 |
|WN        |DEN   |BOS |0.0      |0.0     |29   |9.771185783935497E-5 |
|WN        |LAX   |LAS |0.0      |0.0     |213  |7.176767489580209E-4 |
|WN        |LAX   |SAT |0.0      |0.0     |38   |1.2803622751363753E-4|
|WN        |MCO   |CMH |0.0      |0.0     |77   |2.594418294355287E-4 |
|WN        |PIT   |DEN |0.0      |0.0     |21   |7.0756862573326

In [ ]:
from pyspark.sql.functions import when, lit

# Definir tamaño total de la muestra (por ejemplo, 1% del total)
tamaño_muestra_total = int(total * 0.01)

# Establecer un mínimo de registros por partición
minimo_por_particion = 0

# Calcular el tamaño de muestra por partición según su probabilidad
combinaciones_con_tamaño = combinaciones.withColumn(
    "tamaño_muestra",
    round(
        when(
            (col("probabilidad") * tamaño_muestra_total) < minimo_por_particion,
            lit(minimo_por_particion)
        ).otherwise(col("probabilidad") * tamaño_muestra_total)
    ).cast("int")
)

# Ordenar para visualizar mejor
combinaciones_con_tamaño = combinaciones_con_tamaño.orderBy(col("tamaño_muestra").desc())

combinaciones_con_tamaño.show(10, truncate=False)

# Calcular el tamaño final total de la muestra
tamaño_muestra_final = combinaciones_con_tamaño.agg({"tamaño_muestra": "sum"}).collect()[0][0]

print(f"Tamaño total esperado de la muestra final: {tamaño_muestra_final}")

+----------+------+----+---------+--------+-----+---------------------+--------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |tamaño_muestra|
+----------+------+----+---------+--------+-----+---------------------+--------------+
|HA        |HNL   |OGG |0.0      |0.0     |482  |0.0016240384647782447|5             |
|HA        |OGG   |HNL |0.0      |0.0     |505  |0.001701534076168078 |5             |
|HA        |LIH   |HNL |0.0      |0.0     |355  |0.001196127914930035 |4             |
|WN        |HOU   |DAL |0.0      |0.0     |374  |0.0012601460286868536|4             |
|HA        |HNL   |LIH |0.0      |0.0     |354  |0.0011927585405217813|4             |
|WN        |DAL   |HOU |0.0      |0.0     |375  |0.0012635154030951074|4             |
|DL        |ATL   |FLL |0.0      |0.0     |297  |0.001000704199251325 |3             |
|OO        |MKE   |ORD |0.0      |0.0     |286  |9.636410807605352E-4 |3             |
|AA        |LAX   |DFW |0.0      |0.0     |

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Unir el tamaño de muestra a cada combinación en el dataset original
df_con_muestra = df.join(
    combinaciones_con_tamaño.select(*vars_particion, "tamaño_muestra"),
    on=vars_particion,
    how="inner"
)

# Crear ventana ordenada por fecha y hora dentro de cada combinación
ventana = Window.partitionBy(*vars_particion).orderBy("FL_DATE", "CRS_DEP_TIME")

# Enumerar vuelos por combinación (ordenados por tiempo)
df_con_muestra = df_con_muestra.withColumn("row_num", row_number().over(ventana))

# Filtrar solo los primeros N registros por combinación
muestra_final = df_con_muestra.filter(col("row_num") <= col("tamaño_muestra"))

# Mostrar la muestra final
muestra_final.show(10, truncate=False)

print(f"Tamaño total de la muestra: {muestra_final.count()}")

+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|FL_DATE   |OP_CARRIER_FL_NUM|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|tamaño_muestra|row_num|
+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|AA        |ALB   |CLT |0.0      |0.0     |2016-01-01|1707             |800         |754.0   |-6.0     |9.0     |803.0     |959.0    |5.0    |1030        |1004.0  |-26.0    |150.0           |130.0              |116.0   |646.0   |1           

# **3. Preparación de los datos**

### Selección de variables importantes para el modelo

Se realizará un análisis de predicción para saber que tanto un avión se retrasará. Por ello se quitarán las columnas que no aportan mucha información para el modelo.

In [ ]:
import pandas as pd

pandas_df = muestra_final.limit(50).toPandas()
display(pandas_df)

,OP_CARRIER,ORIGIN,DEST,CANCELLED,DIVERTED,FL_DATE,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,tamaño_muestra,row_num
0,AA,ALB,CLT,0.0,0.0,2016-01-01,1707,800,754.0,-6.0,...,5.0,1030,1004.0,-26.0,150.0,130.0,116.0,646.0,1,1
1,AA,ATL,CLT,0.0,0.0,2016-01-01,1905,710,708.0,-2.0,...,6.0,824,819.0,-5.0,74.0,71.0,41.0,226.0,2,1
2,AA,ATL,CLT,0.0,0.0,2016-01-01,713,850,852.0,2.0,...,11.0,1009,957.0,-12.0,79.0,65.0,39.0,226.0,2,2
3,AA,ATL,LAX,0.0,0.0,2016-01-01,2515,715,715.0,0.0,...,11.0,915,922.0,7.0,300.0,307.0,284.0,1947.0,1,1
4,AA,AUS,CLT,0.0,0.0,2016-01-01,765,1346,1342.0,-4.0,...,10.0,1716,1706.0,-10.0,150.0,144.0,121.0,1032.0,1,1
5,AA,AUS,DFW,0.0,0.0,2016-01-01,1015,620,620.0,0.0,...,10.0,724,724.0,0.0,64.0,64.0,36.0,190.0,3,1
6,AA,AUS,DFW,0.0,0.0,2016-01-01,1595,645,640.0,-5.0,...,9.0,749,735.0,-14.0,64.0,55.0,36.0,190.0,3,2
7,AA,AUS,DFW,0.0,0.0,2016-01-01,1161,800,756.0,-4.0,...,5.0,909,846.0,-23.0,69.0,50.0,35.0,190.0,3,3
8,AA,AUS,LAX,0.0,0.0,2016-01-01,145,705,659.0,-6.0,...,9.0,828,818.0,-10.0,203.0,199.0,177.0,1242.0,1,1
9,AA,BNA,CLT,0.0,0.0,2016-01-01,1903,629,622.0,-7.0,...,7.0,851,842.0,-9.0,82.0,80.0,47.0,328.0,1,1


In [ ]:
# Se crea una nueva columna para verificar si
delayed_time = 10
df_muestra = muestra_final.withColumn("DELAYED", when(df.ARR_DELAY > delayed_time, 1).otherwise(0))

In [ ]:
# Variables a descartar
cols_to_drop = ["tamaño_muestra", "row_num", "OP_CARRIER_FL_NUM", "DEP_TIME","DEP_DELAY", "TAXI_OUT",
                "WHEELS_OFF", "WHEELS_ON", "TAXI_IN" , "ARR_TIME", "ARR_DELAY", "DIVERTED", "ACTUAL_ELAPSED_TIME", "AIR_TIME"]

# Variables importantes para el modelo
selected_cols = [
    "FL_DATE", "OP_CARRIER", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "CANCELLED", "ARR_DELAY", "DELAYED"
]

# Realizando la selección de columnas importantes como datos de entrenamiento
df_raw = df_muestra.select(*selected_cols)
df_raw.show()

+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|   FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|CANCELLED|ARR_DELAY|DELAYED|
+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|2016-01-01|        AA|   ALB| CLT|         800|        1030|           150.0|   646.0|      0.0|    -26.0|      0|
|2016-01-01|        AA|   ATL| CLT|         710|         824|            74.0|   226.0|      0.0|     -5.0|      0|
|2016-01-01|        AA|   ATL| CLT|         850|        1009|            79.0|   226.0|      0.0|    -12.0|      0|
|2016-01-01|        AA|   ATL| LAX|         715|         915|           300.0|  1947.0|      0.0|      7.0|      0|
|2016-01-01|        AA|   AUS| CLT|        1346|        1716|           150.0|  1032.0|      0.0|    -10.0|      0|
|2016-01-01|        AA|   AUS| DFW|         620|         724|           

### Transformación de variables

In [ ]:
from pyspark.sql.functions import dayofweek, month
from pyspark.ml.feature import StringIndexer

# Extrayendo el día, mes y eliminando la columna original de fecha
df_transformed = df_raw.withColumn("FL_DAY_OF_WEEK", dayofweek("FL_DATE")) \
                     .withColumn("FL_MONTH", month("FL_DATE")) \
                     .drop("FL_DATE")

# Codificaando las variables categóricas
indexers = [
    StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_T"),
    StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_T"),
    StringIndexer(inputCol="DEST", outputCol="DEST_T")
]

for indexer in indexers:
    df_transformed = indexer.fit(df_transformed).transform(df_transformed)

# Eliminando las columnas de las variables categóricas originales
cols_to_drop = ["OP_CARRIER", "ORIGIN", "DEST"]
df_transformed = df_transformed.drop(*cols_to_drop) # Using * to unpack the list of column names

df_transformed.show()

+------------+------------+----------------+--------+---------+---------+-------+--------------+--------+------------+--------+------+
|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|CANCELLED|ARR_DELAY|DELAYED|FL_DAY_OF_WEEK|FL_MONTH|OP_CARRIER_T|ORIGIN_T|DEST_T|
+------------+------------+----------------+--------+---------+---------+-------+--------------+--------+------------+--------+------+
|         800|        1030|           150.0|   646.0|      0.0|    -26.0|      0|             6|       1|         1.0|    68.0|   9.0|
|         710|         824|            74.0|   226.0|      0.0|     -5.0|      0|             6|       1|         1.0|     0.0|   9.0|
|         850|        1009|            79.0|   226.0|      0.0|    -12.0|      0|             6|       1|         1.0|     0.0|   9.0|
|         715|         915|           300.0|  1947.0|      0.0|      7.0|      0|             6|       1|         1.0|     0.0|   2.0|
|        1346|        1716|           150.0|  1032.0|  

In [ ]:
# Guardando dataset en un archivo paquet para su posterior utilización
df_transformed.write.mode("overwrite").parquet("./df_transformed.parquet")

In [ ]:
# Lectura del archivo que contiene el dataset de pyspark
df_transformed = spark.read.parquet("./df_transformed.parquet")

In [ ]:
# Imprime los valores resultantes de la limpieza y transformación de los datos
df_transformed.printSchema()

root
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- DELAYED: integer (nullable = true)
 |-- FL_DAY_OF_WEEK: integer (nullable = true)
 |-- FL_MONTH: integer (nullable = true)
 |-- OP_CARRIER_T: double (nullable = true)
 |-- ORIGIN_T: double (nullable = true)
 |-- DEST_T: double (nullable = true)



In [ ]:
# Imprime un resumen del dataframe con la limpieza de todos los datos
df_transformed.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,FL_MONTH,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250
1,mean,938.4142222222222,1163.2493333333334,137.42177777777778,773.6871111111111,0.0,-4.585333333333334,0.11911111111111111,5.996888888888889,1.0,2.416888888888889,26.475555555555555,26.477777777777778
2,stddev,320.65441223776276,309.85682024878986,71.16605726701093,560.1377044875112,0.0,27.545379640307743,0.32399121560794986,0.26419523197213174,0.0,2.425131335203217,34.7800377392978,34.51527644518503
3,min,10,1,32.0,67.0,0.0,-44.0,0,1,1,0.0,0.0,0.0
4,max,2230,2347,412.0,2704.0,0.0,617.0,1,7,1,11.0,199.0,198.0


### Limpieza de datos

In [ ]:
df_transformed.groupBy("CANCELLED").count().show()  # Cuenta cuantos vuelos cancelados hay
df_transformed.groupBy("FL_MONTH").count().show()   # Cuenta cuantos meses diferentes hay

+---------+-----+
|CANCELLED|count|
+---------+-----+
|      0.0| 2250|
+---------+-----+

+--------+-----+
|FL_MONTH|count|
+--------+-----+
|       1| 2250|
+--------+-----+



In [ ]:
# Se elimina la columna de Cancelled y del mes ya que unicamente se cuenta con datos de un solo mes y no existen vuelos cancelasdos
drop_cols = ["CANCELLED", "FL_MONTH"]
df_transformed_2 = df_transformed.drop(*drop_cols)

In [ ]:
#Se eliminan registros con valores nulos
df_clean = df_transformed_2.dropna()

#Se eliminan columnas con valores nulos
df_clean = df_clean.na.drop()

#Se eliminan registros duplicados
df_clean = df_clean.dropDuplicates()

In [ ]:
df_clean.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250
1,mean,938.4142222222222,1163.2493333333334,137.42177777777778,773.6871111111111,-4.585333333333334,0.11911111111111111,5.996888888888889,2.416888888888889,26.475555555555555,26.477777777777778
2,stddev,320.65441223776276,309.85682024878975,71.1660572670109,560.13770448751,27.54537964030771,0.32399121560794997,0.2641952319721319,2.4251313352032167,34.78003773929781,34.51527644518501
3,min,10,1,32.0,67.0,-44.0,0,1,0.0,0.0,0.0
4,max,2230,2347,412.0,2704.0,617.0,1,7,11.0,199.0,198.0


### Balanceo del dataset

In [ ]:
df_clean.groupBy("DELAYED").count().show()   # Cuenta cuantos vuelos retrasados hay (Presencia de dataset desbalanceado)

+-------+-----+
|DELAYED|count|
+-------+-----+
|      1|  268|
|      0| 1982|
+-------+-----+



Esto indica que estamos trabajando con un dataset desbalanceado, por lo que debemos utlizar métodos para balancear el dataset.

In [ ]:
# Objetivo de balanceo
target_size = 1125

# Separar las clases
df_majority = df_clean.filter(col("DELAYED") == 0)
df_minority = df_clean.filter(col("DELAYED") == 1)

# Submuestreo de clase mayoritaria (clase 0)
df_majority_sampled = df_majority.sample(False, fraction=target_size / df_majority.count(), seed=42)

# Sobremuestreo de clase minoritaria (clase 1)
ratio = int(target_size / df_minority.count())
df_minority_oversampled = df_minority
for _ in range(ratio - 1):
    df_minority_oversampled = df_minority_oversampled.union(df_minority)

# Agregar una fracción adicional si no es exacto
remaining = target_size - df_minority_oversampled.count()
if remaining > 0:
    df_minority_oversampled = df_minority_oversampled.union(
        df_minority.sample(withReplacement=True, fraction=remaining / df_minority.count(), seed=42)
    )

# Unir los datasets balanceados
df_balanced = df_majority_sampled.union(df_minority_oversampled)

# Verificación
df_balanced.groupBy("DELAYED").count().show()

+-------+-----+
|DELAYED|count|
+-------+-----+
|      0| 1161|
|      1| 1138|
+-------+-----+



In [ ]:
# Resumen final del dataset balanceado
df_balanced.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299
1,mean,953.0395824271422,1166.6511526750762,137.10874293170943,765.6124401913876,13.59112657677251,0.4949978251413658,5.985210961287517,2.4597651152675075,27.86211396259243,27.698999565028274
2,stddev,342.5003283704341,325.41147429039086,74.6028673825625,575.9769328633676,48.34624385199168,0.5000837505964232,0.3360402530152474,2.332372201012667,37.06304046787446,35.314622866550636
3,min,15,1,32.0,67.0,-44.0,0,1,0.0,0.0,0.0
4,max,2230,2347,412.0,2704.0,617.0,1,7,11.0,199.0,198.0


# **4. Prepraración del conjunto de entrenamiento y prueba**

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "OP_CARRIER_T","ORIGIN_T","DEST_T",
        "CRS_DEP_TIME","CRS_ARR_TIME","CRS_ELAPSED_TIME","DISTANCE",
        "FL_DAY_OF_WEEK",
        "ARR_DELAY","DELAYED"
    ],
    outputCol="features"
)

df_vector = assembler.transform(df_balanced)

In [ ]:
# Separación de datos de entrenamiento y prueba
spark.conf.set("spark.sql.shuffle.partitions", "200")       # Se define el valor por default del número de ejecutores
df_train, df_val_test = df_vector.randomSplit([0.7, 0.3], seed=42)
df_val, df_test = df_val_test.randomSplit([0.5, 0.5], seed=42)

# Impresion del tamaño de las particiones
print(f"""Total data: {df_vector.count()}
Training data: {df_train.count()}
Validation data: {df_val.count()}
Test data: {df_test.count()}""")

 Total data: 2299
Training data: 1638 
Validation data: 332 
Test data: 329



# **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

### Modelo Supervisado - Regresión Logística

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="DELAYED")
lr_model = lr.fit(train_df)

NameError: name 'train_df' is not defined

### Modelo No Supervisado -